# RSA密码破译

`Import the necesssary module`

In [7]:
import gmpy2
from Crypto.Util.number import *
import owiener
import sage

`Load data`

In [8]:
def load_data(datanum):
    with open(file="./test/data%d" % (datanum), mode='r') as file:
        n, e, c = file.read(256), file.read(256), file.read(256)
        n, e, c = int(n, 16), int(e, 16), int(c, 16)
        return n, e, c


`Attack RSA`

In [9]:
def attack_broadcast(e,lst_c,lst_n):
    M=1
    for n in lst_n:
        M=M*n
    lst_M=[]
    for n in lst_n:
        lst_M+=[M//n]
    m_e=M
    for i in range(len(lst_c)):
        m_e+=lst_c[i]*lst_M[i]*inverse(lst_M[i],lst_n[i])
    m_e=m_e%M
    m=gmpy2.iroot(m_e,e)
    i=0
    while not m[1]:
        m_e+=M
        i+=1
        m=gmpy2.iroot(m_e,e)
    return m

In [10]:
lst_n,lst_c=[],[]
for i in range(21):
    n,e,c=load_data(i)
    if e==5:
        lst_n+=[n]
        lst_c+=[c]
print(hex(attack_broadcast(5,lst_c,lst_n)[0]))

0x9876543210abcdef0000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000007420697320612066


In [11]:
def fmt(n):
    for i in range(100000):
        m=gmpy2.iroot(n+i*i,2)
        if m[1]:
            return m[0]-i,m[0]+i
    return 0

n,e,c=load_data(8)
p,q=fmt(n)   

In [12]:
def decode(p,q,e,c):
    d=inverse(e,(p-1)*(q-1))
    m=gmpy2.powmod(c,d,p*q)
    print(hex(m))
decode(p,q,e,c)

0x9876543210abcdef00000008000000000000000000000000000000000000000000000000000000000000000000000000000000000000000077696c6c20676574


In [13]:
def attack_gcd(i,j):
    n1,e1,c1=load_data(i)
    n2,e2,c2=load_data(j)
    p1=GCD(n1,n2)
    q1=n1//p1
    q2=n2//p1
    print(i,":")
    decode(p1,q1,e1,c1)
    print(j,":")
    decode(p1,q2,e2,c2)

for i in range(21):
    for j in range(21):
        n1,e1,c1=load_data(i)
        n2,e2,c2=load_data(j)
        if GCD(n1,n2)!=1 and i<j and n1!=n2:
            attack_gcd(i,j)

2 :
0x9876543210abcdef0000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000006d204120746f2042
19 :
0x9876543210abcdef0000000b00000000000000000000000000000000000000000000000000000000000000000000000000000000000000002e20496d6167696e


In [14]:
n1,e1,c1=load_data(11)
n2,e2,c2=load_data(14)

def attack_mod(e1,e2,c1,c2,n):  
    e1, e2, c1, c2, n=int(e1),int(e2),int(c1),int(c2),int(n)
    s = gmpy2.gcdext(e1, e2)
    s1 = s[1]
    s2 = s[2]
    if s1 < 0:
        s1 = - s1
        c1 = gmpy2.invert(c1, n)
    elif s2 < 0:
        s2 = - s2
        c2 = gmpy2.invert(c2, n)
    m = (pow(c1,s1,n) * pow(c2 ,s2 ,n)) % n
    return hex(m)
attack_mod(e1,e2,c1,c2,n1)

'0x9876543210abcdef0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004d79207365637265'

In [ ]:
m_0=0x9876543210abcdef00000003000000000000000000000000000000000000000000000000000000000000000000000000000000000000000079696e67206f6620
m_6=0x9876543210abcdef000000040000000000000000000000000000000000000000000000000000000000000000000000000000000000000000416c626572742045
m_17=0x9876543210abcdef000000020000000000000000000000000000000000000000000000000000000000000000000000000000000000000000616d6f7573207361